# 02_고급 기능

## 1. OpenAI 라이브러리 설치 확인

In [ ]:
# !pip install openai python-dotenv

## 2. OpenAI 정보 읽기

In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
dotenv_path = find_dotenv(filename='././.env')
load_dotenv(dotenv_path=dotenv_path)

openai.api_type = "azure"
openai.api_version = os.getenv("OPENAI_API_VERSION","").strip()

API_KEY = os.getenv("OPENAI_API_KEY","").strip()
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = RESOURCE_ENDPOINT

model=os.getenv('DEPLOYMENT_NAME')

## 3. Chain of Thought

In [2]:
def call_openai_api(messages, max_token=100, stop=None, n=1, temperature=0.1):
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_token,
        stop=stop,
        n=n,
    )
    return response

#### This prompt gets wrong answer

In [3]:
PROMPT_ZERO_SHOT = """A juggler can juggle 16 balls. Half of the balls are golf balls,
and half of the golf balls are blue. How many blue golf balls are there?
"""
messages=[
    {"role": "user", "content": PROMPT_ZERO_SHOT},
    {"role": "assistant", "content": "The answer (arabic numerals) is "},
]

response = call_openai_api(messages, temperature=0, max_token=100)

print(response['choices'][0]["message"]["content"])

8


#### With CoT, the answer is correct

In [4]:
PROMPT_ZERO_SHOT_CoT = """Q: A juggler can juggle 16 balls. Half of the balls are golf balls,
and half of the golf balls are blue. How many blue golf balls are
there?
A: Let's think step by step.
"""

messages=[
    {"role": "user", "content": PROMPT_ZERO_SHOT_CoT},
]

response = call_openai_api(messages, temperature=0, max_token=100)

print(response['choices'][0]["message"]["content"])

Step 1: We know that half of the balls are golf balls. So, we can divide the total number of balls by 2 to find the number of golf balls.

16 balls / 2 = 8 golf balls

Step 2: We also know that half of the golf balls are blue. So, we can divide the number of golf balls by 2 to find the number of blue golf balls.

8 golf balls / 2 = 4 blue golf balls

Therefore,


#### Can be wrong answer with few-shot learning

In [5]:
messages=[
    {"role": "system", "content": "Generate next conversation."},
    {"role": "user", "content": "Q: Roger has 5 tennis balss. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does Roger have now?"},
    {"role": "assistant", "content": "A: The answer is 11."},
    {"role": "user", "content": "Q: A juggler can juggle 16 balls. Half of the balls are golf balls and half of the golf balls are blue. How many blue golf balls are there?"},
]

response = call_openai_api(messages, temperature=0, max_token=100)

print(response['choices'][0]["message"]["content"])

A: If half of the balls are golf balls and half of the golf balls are blue, then there are 8 golf balls in total and 4 of them are blue.


#### With few-show CoT, the answer is correct

In [6]:
messages=[
    {"role": "user", "content": """Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?"""},
    {"role": "assistant", "content": """Roger started with 5 balls.\n 2 cans of 3 tennis balls each is 6 tennis balls.\n 5 + 6 = 11. \n The answer is 11."""},
    {"role": "user", "content": """A juggler can juggle 16 balls. Half of the balls are golf balls, and half of the golf balls are blue. How many blue golf balls are there?"""},
]

response = call_openai_api(messages, temperature=0, max_token=100)

print(response['choices'][0]["message"]["content"])

If half of the balls are golf balls, then there are 16/2 = <<16/2=8>>8 golf balls.
If half of the golf balls are blue, then there are 8/2 = <<8/2=4>>4 blue golf balls. Answer: \boxed{4}.


## 4. SQL

### SQL 생성

In [7]:
prompt = "### Postgres SQL tables, with their properties:\n#\n# Employee(id, name, department_id)\n# Department(id, name, address)\n# Salary_Payments(id, employee_id, amount, date)\n#\n### A query to list the names of the departments which employed more than 10 employees in the last 3 months\n\n query: "
print(prompt)

### Postgres SQL tables, with their properties:
#
# Employee(id, name, department_id)
# Department(id, name, address)
# Salary_Payments(id, employee_id, amount, date)
#
### A query to list the names of the departments which employed more than 10 employees in the last 3 months

 query: 


In [8]:
response = openai.ChatCompletion.create(
  engine=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content": prompt}],
  temperature=0,
  max_tokens=150,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  stop=["#",";"])
print(response['choices'][0]['message']['content'])

SELECT d.name
FROM Department d
WHERE d.id IN (
    SELECT e.department_id
    FROM Employee e
    WHERE e.id IN (
        SELECT sp.employee_id
        FROM Salary_Payments sp
        WHERE sp.date >= CURRENT_DATE - INTERVAL '3 months'
        GROUP BY sp.employee_id
        HAVING COUNT(sp.employee_id) > 10
    )
)


### SQL 해석

In [9]:
code = "Can you explain what does this code do?\n#\n# ###\n\
   Code:\n\
   SELECT d.name FROM Department d JOIN Employee e ON d.id = e.department_id WHERE e.id IN (SELECT employee_id FROM Salary_Payments WHERE date > now() - interval '3 months') GROUP BY d.name HAVING COUNT(*) > 10\n#\n#\
   Answer:\n# "
print(code)

Can you explain what does this code do?
#
# ###
   Code:
   SELECT d.name FROM Department d JOIN Employee e ON d.id = e.department_id WHERE e.id IN (SELECT employee_id FROM Salary_Payments WHERE date > now() - interval '3 months') GROUP BY d.name HAVING COUNT(*) > 10
#
#   Answer:
# 


In [10]:
response = openai.ChatCompletion.create(
  engine=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content": code}],
  temperature=0,
  max_tokens=500,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  stop=["#",";"])
print(response['choices'][0]['message']['content'])

This code is a SQL query that retrieves the names of departments that have more than 10 employees who have received salary payments in the last 3 months. 

Here is a breakdown of the code:

1. The query starts with the SELECT statement, which specifies that we want to retrieve the "name" column from the "Department" table.

2. The FROM clause specifies that we are selecting from the "Department" table.

3. The JOIN keyword is used to combine the "Department" table with the "Employee" table. The ON clause specifies the condition for the join, which is that the "id" column in the "Department" table must match the "department_id" column in the "Employee" table.

4. The WHERE clause filters the results based on a condition. In this case, it checks if the "id" column in the "Employee" table is present in the subquery.

5. The subquery selects the "employee_id" column from the "Salary_Payments" table where the "date" column is greater than the current date minus 3 months.

6. The GROUP BY cl

### SQL 실제 사용 사례

In [11]:
query = '''
Given the following SQL tables, your job is to write queries given a user’s request.

CREATE TABLE Orders (
  OrderID int,
  CustomerID int,
  OrderDate datetime,
  OrderTime varchar(8),
  PRIMARY KEY (OrderID)
);

CREATE TABLE OrderDetails (
  OrderDetailID int,
  OrderID int,
  ProductID int,
  Quantity int,
  PRIMARY KEY (OrderDetailID)
);

CREATE TABLE Products (
  ProductID int,
  ProductName varchar(50),
  Category varchar(50),
  UnitPrice decimal(10, 2),
  Stock int,
  PRIMARY KEY (ProductID)
);

CREATE TABLE Customers (
  CustomerID int,
  FirstName varchar(50),
  LastName varchar(50),
  Email varchar(100),
  Phone varchar(20),
  PRIMARY KEY (CustomerID)
);
'''

In [12]:
request1 = "Write a SQL query which computes the average total order value for all orders on 2023-04-01."

response = openai.ChatCompletion.create(
  engine=model,
  messages = [{"role":"system", "content":query},
               {"role":"user","content": request1}],
  temperature=0,
  max_tokens=500,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  stop=["#",";"])
print(response['choices'][0]['message']['content'])

SELECT AVG(total_order_value) AS average_total_order_value
FROM (
  SELECT o.OrderID, SUM(od.Quantity * p.UnitPrice) AS total_order_value
  FROM Orders o
  INNER JOIN OrderDetails od ON o.OrderID = od.OrderID
  INNER JOIN Products p ON od.ProductID = p.ProductID
  WHERE o.OrderDate = '2023-04-01'
  GROUP BY o.OrderID
) AS subquery


In [13]:
request2 = "한글로 번역:create the 5 questions to answer from tables and show the query"

response = openai.ChatCompletion.create(
  engine=model,
  messages = [{"role":"system", "content":query},
               {"role":"user","content": request2}],
  temperature=0,
  max_tokens=500,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  stop=["#",";"])
print(response['choices'][0]['message']['content'])

1. 주문한 고객의 성명과 이메일을 조회하시오.
```sql
SELECT FirstName, LastName, Email
FROM Customers
WHERE CustomerID IN (
  SELECT CustomerID
  FROM Orders
)
```

2. 주문한 상품의 카테고리와 수량을 조회하시오.
```sql
SELECT Category, Quantity
FROM Products
JOIN OrderDetails ON Products.ProductID = OrderDetails.ProductID
```

3. 주문일자가 특정 날짜 이후인 주문의 주문번호와 주문일자를 조회하시오.
```sql
SELECT OrderID, OrderDate
FROM Orders
WHERE OrderDate > 'YYYY-MM-DD'
```

4. 주문한 상품의 재고량이 특정 수량보다 적은 상품의 상품명과 재고량을 조회하시오.
```sql
SELECT ProductName, Stock
FROM Products
WHERE Stock < 'quantity'
```

5. 특정 고객의 주문한 상품의 주문일자와 주문시간을 조회하시오.
```sql
SELECT OrderDate, OrderTime
FROM Orders
WHERE CustomerID = 'customerID'
```


## 5. Python algorithm

### Generate problem statement

In [14]:
response = openai.ChatCompletion.create(
    engine=model,
    messages=[
        {
            "role": "user", "content": "Generate a coding challenge problem statement with description, "
                                       "input format, output format, constraints, and example inputs and "
                                       "outputs."
        },
        {
            "role": "assistant",
            "content": "Generate a set of input/output examples with inputs and outputs that can be parsed "
                       "using ast.literal_eval() in Python. The input/output format must adhere to the "
                       "following structure:\n\n1. In: [input_1]\nOut: [output_1]\n2. In: [input_2]\nOut: ["
                       "output_2]\n...\n\nPlease ensure that 'In:' and 'Out:' keywords are only used to "
                       "denote the input and output examples in the response. Additionally, provide examples "
                       "with a variety of Python literals, including tuples, dictionaries, lists, booleans, "
                       "and numerical values. Verify that the generated statement contains the 'In:' and "
                       "'Out:' test cases before returning the result."
        }
    ],
    temperature=1,
    max_tokens=400,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
)

In [15]:
problem_statement = response.choices[0].message.content.strip()
print(problem_statement)

Problem Statement:
Implement a function that takes a list of tuples as input and returns a list of dictionaries. Each tuple in the input list represents a person's information, and the dictionaries in the output list should contain the same information but structured differently.

Input Format:
A list of tuples, where each tuple contains the details of a person. The details include the person's name (a string), age (an integer), and favorite color (a string).

Output Format:
A list of dictionaries, where each dictionary represents a person's information. The dictionaries should have the following key-value pairs:
- 'name': the person's name (a string)
- 'age': the person's age (an integer)
- 'color': the person's favorite color (a string)

Constraints:
- The input list will always contain at least one tuple (i.e., one person's information).
- The name will be a non-empty string (length <= 100).
- The age will be a positive integer (1 <= age <= 100).
- The color will be a non-empty stri

### Generate algorithm

In [16]:
response = openai.ChatCompletion.create(
    engine=model,
    messages=[{
        "role": "user",
        "content": f"{problem_statement}\n\nPlease provide a Python function to solve the above problem. The "
                   "generated code should be complied without any additional modification. So, no comments, "
                   "markdown, code fence, etc in the response."
    }],
    temperature=0.3,
    max_tokens=350,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
)

In [17]:
solution_code = response.choices[0].message.content.strip()
print(solution_code)

def convert_to_dict(persons):
    result = []
    for person in persons:
        person_dict = {'name': person[0], 'age': person[1], 'color': person[2]}
        result.append(person_dict)
    return result
